# Reliability Experiments

## Imports

In [1]:
import os 
import sys
from glob import glob

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics

## Helper functions

In [2]:
def load_experiment(experiment_name):

    ### process (combined) data frame
    experiment_files = list(sorted(glob(f"{experiment_name}/*_metrics.csv")))

    df = None
    for i, experiment_file in enumerate(experiment_files):
        df_exp = pd.read_csv(experiment_file)
        if df is None:
            df = df_exp.copy()
        else:
            df = pd.concat((df, df_exp), axis=0)

    if df is None:
        print(f"No *_metrics.csv found for experiment_name={experiment_name}")
    
    ### assign experiment name
    df = df.assign(experiment_name = experiment_name)
    
    return df


def load_experiments(experiment_names):
    
    df = None
    for i, experiment_name in enumerate(experiment_names):
        df_exp = load_experiment(experiment_name)
        if df is None:
            df = df_exp.copy()
        else:
            df = pd.concat((df, df_exp), axis=0, ignore_index=True)
    return df
    

## Load result files

In [3]:
# define which experiments to load
experiment_names = [
    
    # TreeRing
    'TreeRing_InPaint_ReplaceBG',
    'TreeRing_InPaint_ReplaceBG_attack_seed_2',
    'TreeRing_InPaint_ReplaceBG_attack_seed_3',
    'TreeRing_InPaint_ReplaceBG_attack_seed_4',
    'TreeRing_InPaint_ReplaceBG_attack_seed_5',

    
    # StegaStamp
    'StegaStamp_InPaint_ReplaceBG',
    'StegaStamp_InPaint_ReplaceBG_attack_seed_2',
    'StegaStamp_InPaint_ReplaceBG_attack_seed_3',
    'StegaStamp_InPaint_ReplaceBG_attack_seed_4',
    'StegaStamp_InPaint_ReplaceBG_attack_seed_5',

    
    # StableSig
    'StableSig_InPaint_ReplaceBG',
    'StableSig_InPaint_ReplaceBG_attack_seed_2',
    'StableSig_InPaint_ReplaceBG_attack_seed_3',
    'StableSig_InPaint_ReplaceBG_attack_seed_4',
    'StableSig_InPaint_ReplaceBG_attack_seed_5',
    
    # Invisible
    'Invisible_InPaint_ReplaceBG',
    'Invisible_InPaint_ReplaceBG_attack_seed_2',
    'Invisible_InPaint_ReplaceBG_attack_seed_3',
    'Invisible_InPaint_ReplaceBG_attack_seed_4',
    'Invisible_InPaint_ReplaceBG_attack_seed_5',
    
]

In [4]:
# load the experiment result files
df_ = load_experiments(experiment_names)

# display number of successful prompts 
df_.groupby("experiment_name").count()[['prompt_index']]

,prompt_index
experiment_name,
Invisible_InPaint_ReplaceBG,1000
Invisible_InPaint_ReplaceBG_attack_seed_2,1000
Invisible_InPaint_ReplaceBG_attack_seed_3,1000
Invisible_InPaint_ReplaceBG_attack_seed_4,1000
Invisible_InPaint_ReplaceBG_attack_seed_5,1000
StableSig_InPaint_ReplaceBG,1000
StableSig_InPaint_ReplaceBG_attack_seed_2,1000
StableSig_InPaint_ReplaceBG_attack_seed_3,1000
StableSig_InPaint_ReplaceBG_attack_seed_4,1000


## Compute additional metrics

In [5]:
# Assign watermark type
df_ = df_.assign(wm_type = [_.split('_')[0] for _ in df_.experiment_name])

# Compute success rate based on (p > 0.05) and (Bit Acc < 24/32)
df_ = df_.assign(w_bit_acc_success = df_.w_bit_acc.le(24/32))
df_ = df_.assign(w_p_success = df_.w_p.ge(0.05))

## Compute Average Watermark Removal Metrics (Table 7)

In [6]:
# select watermark removal metric columns
use_columns = ['w_p', 'w_bit_acc', 'w_pct_mask', 'experiment_name']

# extract watermark removal metrics
df = df_[use_columns].groupby("experiment_name").mean()
df = df.loc[experiment_names]

# save the dataframe to csv
# save_as = "Table_07-average_watermark_removal_metrics_by_run.csv"
# df.to_csv(save_as)
# print(f"[+] {save_as}")

# show dataframe
# df

## Compute Average Watermark Removal Metrics (Table 7)

In [7]:
# define watermark types
wm_types = ["TreeRing", "StegaStamp", "StableSig", "Invisible"]

# select watermark removal metric columns
use_columns = ['w_p', 'w_bit_acc', 'w_pct_mask', 'experiment_name']

# define threshold
min_pct_mask = 0.00
max_pct_mask = 1.00

# store dfs 
dfs = dict()

# loop over watermark types
for i,wm_type in enumerate(wm_types):

    # filter results by {wm_type}_InPaint_ReplaceBG
    df_experiment = df_[df_.experiment_name.isin([f"{wm_type}_InPaint_ReplaceBG"])]

    # filter prompts s.t. (mask pct > 50)
    df_experiment_filter = df_experiment[
        (df_experiment.w_pct_mask >= min_pct_mask)
      & (df_experiment.w_pct_mask <= max_pct_mask)
    ] 
    good_prompts = list(df_experiment_filter.prompt_index.unique())
    # print(f"wm_type={wm_type}, n_good_prompts={len(good_prompts)}")
    
    # filter all results by watermark type
    df_wm = df_[df_.wm_type.isin([wm_type])]
    
    # filter all results based on good prompts
    df_wm_good = df_wm[df_wm.prompt_index.isin(good_prompts)]
    df_wm_good = df_wm_good[use_columns].groupby("experiment_name").mean()
    df_wm_good = df_wm_good.loc[[_ for _ in experiment_names if _.startswith(wm_type)]]
    
    # compute statistics
    df_wm_good = df_wm_good.T.assign(mean=df_wm_good.mean()).T
    df_wm_good = df_wm_good.T.assign(std=df_wm_good.std()).T
    
    
    # save scores
    save_as = f"Table_07a-average_watermark_removal_metrics_by_run_{wm_type}_w_pct_mask_le_{min_pct_mask:0.2f}_ge_{max_pct_mask:0.2f}.csv"
    df_wm_good.to_csv(save_as)
    print(f"[+] {save_as}")
    
    # store df
    dfs[wm_type] = df_wm_good

[+] Table_07a-average_watermark_removal_metrics_by_run_TreeRing_w_pct_mask_le_0.00_ge_1.00.csv
[+] Table_07a-average_watermark_removal_metrics_by_run_StegaStamp_w_pct_mask_le_0.00_ge_1.00.csv
[+] Table_07a-average_watermark_removal_metrics_by_run_StableSig_w_pct_mask_le_0.00_ge_1.00.csv
[+] Table_07a-average_watermark_removal_metrics_by_run_Invisible_w_pct_mask_le_0.00_ge_1.00.csv


In [8]:
# Tree Ring
dfs["TreeRing"]

,w_p,w_bit_acc,w_pct_mask
experiment_name,,,
TreeRing_InPaint_ReplaceBG,0.099848,NaN,0.612084
TreeRing_InPaint_ReplaceBG_attack_seed_2,0.117374,NaN,0.612084
TreeRing_InPaint_ReplaceBG_attack_seed_3,0.046261,NaN,0.612084
TreeRing_InPaint_ReplaceBG_attack_seed_4,0.135640,NaN,0.612084
TreeRing_InPaint_ReplaceBG_attack_seed_5,0.052061,NaN,0.612084
mean,0.090237,NaN,0.612084
std,0.035444,NaN,0.000000


In [9]:
# StegaStamp
dfs["StegaStamp"]

,w_p,w_bit_acc,w_pct_mask
experiment_name,,,
StegaStamp_InPaint_ReplaceBG,0.163942,0.701040,6.562403e-01
StegaStamp_InPaint_ReplaceBG_attack_seed_2,0.157424,0.707360,6.562403e-01
StegaStamp_InPaint_ReplaceBG_attack_seed_3,0.159509,0.701260,6.562403e-01
StegaStamp_InPaint_ReplaceBG_attack_seed_4,0.166676,0.704900,6.562403e-01
StegaStamp_InPaint_ReplaceBG_attack_seed_5,0.176398,0.698610,6.562403e-01
mean,0.164790,0.702634,6.562403e-01
std,0.006651,0.003101,1.216188e-16


In [10]:
# StableSig
dfs["StableSig"]

,w_p,w_bit_acc,w_pct_mask
experiment_name,,,
StableSig_InPaint_ReplaceBG,0.598234,0.493354,6.494498e-01
StableSig_InPaint_ReplaceBG_attack_seed_2,0.598636,0.496917,6.494498e-01
StableSig_InPaint_ReplaceBG_attack_seed_3,0.598545,0.488708,6.494498e-01
StableSig_InPaint_ReplaceBG_attack_seed_4,0.622329,0.498458,6.494498e-01
StableSig_InPaint_ReplaceBG_attack_seed_5,0.608904,0.492500,6.494498e-01
mean,0.605330,0.493987,6.494498e-01
std,0.009412,0.003437,1.216188e-16


In [11]:
# Invisible
dfs["Invisible"]

,w_p,w_bit_acc,w_pct_mask
experiment_name,,,
Invisible_InPaint_ReplaceBG,NaN,0.506292,6.042053e-01
Invisible_InPaint_ReplaceBG_attack_seed_2,NaN,0.506417,6.042053e-01
Invisible_InPaint_ReplaceBG_attack_seed_3,NaN,0.504969,6.042053e-01
Invisible_InPaint_ReplaceBG_attack_seed_4,NaN,0.505948,6.042053e-01
Invisible_InPaint_ReplaceBG_attack_seed_5,NaN,0.505917,6.042053e-01
mean,NaN,0.505908,6.042053e-01
std,NaN,0.000508,1.216188e-16


## Compute Average Watermark Removal Metrics (Table 7b)

In [12]:
# define watermark types
wm_types = ["TreeRing", "StegaStamp", "StableSig", "Invisible"]

# select watermark removal metric columns
use_columns = ['w_p', 'w_bit_acc', 'w_pct_mask', 'experiment_name']

# define threshold
min_pct_mask = 0.50
max_pct_mask = 1.00

# store dfs 
dfs = dict()

# loop over watermark types
for i,wm_type in enumerate(wm_types):

    # filter results by {wm_type}_InPaint_ReplaceBG
    df_experiment = df_[df_.experiment_name.isin([f"{wm_type}_InPaint_ReplaceBG"])]

    # filter prompts s.t. (mask pct > 50)
    df_experiment_filter = df_experiment[
        (df_experiment.w_pct_mask >= min_pct_mask)
      & (df_experiment.w_pct_mask <= max_pct_mask)
    ] 
    good_prompts = list(df_experiment_filter.prompt_index.unique())
    # print(f"wm_type={wm_type}, n_good_prompts={len(good_prompts)}")
    
    # filter all results by watermark type
    df_wm = df_[df_.wm_type.isin([wm_type])]
    
    # filter all results based on good prompts
    df_wm_good = df_wm[df_wm.prompt_index.isin(good_prompts)]
    df_wm_good = df_wm_good[use_columns].groupby("experiment_name").mean()
    df_wm_good = df_wm_good.loc[[_ for _ in experiment_names if _.startswith(wm_type)]]
    
    # compute statistics
    df_wm_good = df_wm_good.T.assign(mean=df_wm_good.mean()).T
    df_wm_good = df_wm_good.T.assign(std=df_wm_good.std()).T
    
    
    # save scores
    save_as = f"Table_07b-average_watermark_removal_metrics_by_run_{wm_type}_w_pct_mask_le_{min_pct_mask:0.2f}_ge_{max_pct_mask:0.2f}.csv"
    df_wm_good.to_csv(save_as)
    print(f"[+] {save_as}")
    
    # store df
    dfs[wm_type] = df_wm_good

[+] Table_07b-average_watermark_removal_metrics_by_run_TreeRing_w_pct_mask_le_0.50_ge_1.00.csv
[+] Table_07b-average_watermark_removal_metrics_by_run_StegaStamp_w_pct_mask_le_0.50_ge_1.00.csv
[+] Table_07b-average_watermark_removal_metrics_by_run_StableSig_w_pct_mask_le_0.50_ge_1.00.csv
[+] Table_07b-average_watermark_removal_metrics_by_run_Invisible_w_pct_mask_le_0.50_ge_1.00.csv


In [13]:
# Tree Ring
dfs["TreeRing"]

,w_p,w_bit_acc,w_pct_mask
experiment_name,,,
TreeRing_InPaint_ReplaceBG,0.145132,NaN,0.738309
TreeRing_InPaint_ReplaceBG_attack_seed_2,0.173758,NaN,0.738309
TreeRing_InPaint_ReplaceBG_attack_seed_3,0.068085,NaN,0.738309
TreeRing_InPaint_ReplaceBG_attack_seed_4,0.201558,NaN,0.738309
TreeRing_InPaint_ReplaceBG_attack_seed_5,0.077306,NaN,0.738309
mean,0.133168,NaN,0.738309
std,0.052582,NaN,0.000000


In [14]:
# StegaStamp
dfs["StegaStamp"]

,w_p,w_bit_acc,w_pct_mask
experiment_name,,,
StegaStamp_InPaint_ReplaceBG,0.229512,0.629916,0.773829
StegaStamp_InPaint_ReplaceBG_attack_seed_2,0.220441,0.635700,0.773829
StegaStamp_InPaint_ReplaceBG_attack_seed_3,0.223128,0.631751,0.773829
StegaStamp_InPaint_ReplaceBG_attack_seed_4,0.233429,0.634132,0.773829
StegaStamp_InPaint_ReplaceBG_attack_seed_5,0.246965,0.626765,0.773829
mean,0.230695,0.631653,0.773829
std,0.009338,0.003145,0.000000


In [15]:
# StableSig
dfs["StableSig"]

,w_p,w_bit_acc,w_pct_mask
experiment_name,,,
StableSig_InPaint_ReplaceBG,0.598360,0.487518,0.783674
StableSig_InPaint_ReplaceBG_attack_seed_2,0.582936,0.491196,0.783674
StableSig_InPaint_ReplaceBG_attack_seed_3,0.593645,0.482966,0.783674
StableSig_InPaint_ReplaceBG_attack_seed_4,0.629807,0.494965,0.783674
StableSig_InPaint_ReplaceBG_attack_seed_5,0.621813,0.489568,0.783674
mean,0.605312,0.489243,0.783674
std,0.017649,0.003976,0.000000


In [16]:
# Invisible
dfs["Invisible"]

,w_p,w_bit_acc,w_pct_mask
experiment_name,,,
Invisible_InPaint_ReplaceBG,NaN,0.506224,0.739225
Invisible_InPaint_ReplaceBG_attack_seed_2,NaN,0.506848,0.739225
Invisible_InPaint_ReplaceBG_attack_seed_3,NaN,0.505264,0.739225
Invisible_InPaint_ReplaceBG_attack_seed_4,NaN,0.504720,0.739225
Invisible_InPaint_ReplaceBG_attack_seed_5,NaN,0.505632,0.739225
mean,NaN,0.505738,0.739225
std,NaN,0.000740,0.000000
